# Real estate price model workflow

Sam Maurer, Feb 2018; Paul Waddell, June 2018

Python 3.6, intended to be backward compatible with 2.7

In [ ]:
from __future__ import print_function
import warnings;warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

In [ ]:
# Standard to run UrbanSim from the root level of the project directory

import os; os.chdir('../..')

In [ ]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import OLSRegressionStep
import orca

In [ ]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

## Explore the Orca registrations

In [ ]:
orca.list_tables()

In [ ]:
for table_name in orca.list_tables():
    print(table_name.upper())
    print(orca.get_table(table_name).to_frame().columns.tolist())
    print()

In [ ]:
orca.list_broadcasts()

In [ ]:
orca.list_steps()

## Explore the data

Orca doesn't execute code to load the registered objects until it needs to.

In [ ]:
orca.get_table('parcels').to_frame().describe()

In [ ]:
orca.get_table('buildings').to_frame().describe()

## Generate accessibility measures for the price model

The network accessibility metrics are not stored on disk; for now we'll generate them using legacy code.

In [ ]:
orca.run(['initialize_network_small'])

In [ ]:
orca.run(['initialize_network_drive'])

In [ ]:
orca.run(['network_aggregations_drive'])

In [ ]:
orca.run(['network_aggregations_small'])

In [ ]:
orca.list_tables()

In [ ]:
print(orca.get_table('nodesdrive').to_frame().columns.tolist())

In [ ]:
nodesdrive = orca.get_table('nodesdrive').to_frame()

In [ ]:
from scipy.stats import norm
%matplotlib inline

import matplotlib.pyplot as plt, matplotlib.cm as cm, matplotlib.font_manager as fm
import matplotlib.mlab as mlab
import seaborn as sns
sns.set()

In [ ]:
plt.rcParams['figure.figsize']=8,8
%matplotlib inline

sns.set_style("white")
sns.set_style("ticks")
ax = sns.distplot(nodesdrive.med_income_1500, bins=500,  fit=norm, kde=False)

In [ ]:
print('{} nodes have median income zero or below'.format(len(nodes[nodes.med_income_1500 == -1])))

print('Total nodes count {}'.format(len (nodes)))

# Estimate a price model

The basic idea of the parcel template is that we create model steps by _passing arguments to classes_ rather than by writing Python functions and giving them Orca decorators, as we would for a fully custom model.

Much of the functionality for this is already built into UrbanSim and Orca, we'll just need to extend things here and there.

This demo uses a new RegressionStep() class that provides a full model development workflow: estimating a model, registering it with Orca, saving it for future use.

### Specify parameters and pass them to a model object

In [ ]:
m = OLSRegressionStep()
m.tables = ['buildings', 'parcels', 'nodesdrive', 'nodessmall']
m.filters = 'redfin_sale_year > 2010 & residential_units == 1 & med_income_1500 > 0'
m.model_expression = 'np.log1p(res_price_per_sqft) ~ year_built + np.log(med_income_1500) + residential_units_1500 + \
    renters_1500 + jobs_25000'
m.fit()

### If we like it, register it as an Orca step

In [ ]:
model.register()

In [ ]:
orca.list_steps()

### Run the Orca step

In [ ]:
orca.run(['RegressionStep-20180214-210159'])

### Check the fitted values

In [ ]:
df = orca.get_table('buildings').to_frame(['res_price_per_sqft', 'fitted_price'])
df.loc[df.fitted_price > 0].describe()

### BONUS

Running "model.register()" also registered the step with the new ModelManager extention, which saves it to disk so that it can be automatically re-loaded in the future.

The "test" model steps here were estimated earlier and loaded from disk. They're fully functional: we can run them in Orca, inspect the estimation results, etc.

In [ ]:
mm.list_steps()

In [ ]:
rs = mm.get_step('test-1')
type(rs)

In [ ]:
rs.model_expression

In [ ]:
rs.model.report_fit()